<a href="https://colab.research.google.com/github/hpnightowl/owlsbooks/blob/master/model_abalone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
#A Seperate Nootbook For Model
import pandas as pd

ad = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/abalone_data/training_set_label.csv" )

In [97]:
ad.head(5)

Sex  Length  Diameter  ...  Viscera weight  Shell weight  Rings
0   F   0.525     0.400  ...          0.1380         0.208     10
1   F   0.445     0.325  ...          0.1125         0.140      9
2   M   0.710     0.540  ...          0.2610         0.780     18
3   F   0.680     0.580  ...          0.4530         0.600     19
4   M   0.605     0.470  ...          0.2405         0.345     12

[5 rows x 9 columns]

In [98]:
ad.shape

(3132, 9)

In [99]:
ad.isnull().sum()

Sex               0
Length            0
Diameter          0
Height            0
Whole weight      0
Shucked weight    0
Viscera weight    0
Shell weight      0
Rings             0
dtype: int64

In [100]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
ad['Sex'] = le.fit_transform(ad['Sex'])
ad.head()

Sex  Length  Diameter  ...  Viscera weight  Shell weight  Rings
0    0   0.525     0.400  ...          0.1380         0.208     10
1    0   0.445     0.325  ...          0.1125         0.140      9
2    2   0.710     0.540  ...          0.2610         0.780     18
3    0   0.680     0.580  ...          0.4530         0.600     19
4    2   0.605     0.470  ...          0.2405         0.345     12

[5 rows x 9 columns]

In [101]:
X = ad.drop(columns=['Rings'],axis=1)
y =ad[['Rings']]

In [102]:
X.corr()

Sex    Length  ...  Viscera weight  Shell weight
Sex             1.000000 -0.017071  ...       -0.009312     -0.015721
Length         -0.017071  1.000000  ...        0.901679      0.900571
Diameter       -0.021079  0.986523  ...        0.898299      0.908248
Height         -0.015726  0.890674  ...        0.859091      0.882061
Whole weight   -0.000659  0.924719  ...        0.965639      0.957100
Shucked weight  0.018821  0.898828  ...        0.933097      0.887878
Viscera weight -0.009312  0.901679  ...        1.000000      0.908051
Shell weight   -0.015721  0.900571  ...        0.908051      1.000000

[8 rows x 8 columns]

In [103]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.4,random_state =40)

In [104]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

hp = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/abalone_data/training_set_label.csv" )
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
hp['Sex'] = le.fit_transform(hp['Sex'])

data  = hp # as to save a copy for me:)
features = data.drop('Rings', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, data['Rings'], random_state=40)

# Average CV score on the training set was: -4.405071512735888
exported_pipeline = make_pipeline(
    PCA(iterated_power=9, svd_solver="randomized"),
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    ExtraTreesRegressor(bootstrap=False, max_features=0.9000000000000001, min_samples_leaf=9, min_samples_split=12, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 40)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

In [105]:
test_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/abalone_data/testing_set_label.csv')
le = LabelEncoder()
test_data['Sex'] = le.fit_transform(test_data['Sex'])

results = exported_pipeline.predict(test_data)

In [106]:
# To create Dataframe of predicted value with particular respective index
res = pd.DataFrame(results) #preditcions are nothing but the final predictions of your model on input features of your new unseen test data
res.index = test_data.index # its important for comparison. Here "test_new" is your new test dataset
res.columns = ["prediction"]

# To download the csv file locally
from google.colab import files
res.to_csv('hpi.csv')         
files.download('hpi.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>